In [16]:
#顧客情報から融資できるか予測
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import warnings
import mglearn
# 実行上問題ない注意は非表示にする
warnings.filterwarnings('ignore') 

In [17]:
#csvファイルの読み込み
df = pd.read_csv("data/bank_01.csv")

In [18]:
#ワンホットエンコーディング
data_dummies = pd.get_dummies(df)
data_dummies.columns
data_dummies.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,30,487,2,999,0,-1.8,92.893,-46.2,1.313,5099.1,...,1,0,0,0,0,0,1,0,1,0
1,39,346,4,999,0,1.1,93.994,-36.4,4.855,5191.0,...,1,0,0,0,0,0,1,0,1,0
2,25,227,1,999,0,1.4,94.465,-41.8,4.962,5228.1,...,0,0,0,0,1,0,1,0,1,0
3,38,17,3,999,0,1.4,94.465,-41.8,4.959,5228.1,...,1,0,0,0,0,0,1,0,1,0
4,47,58,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,...,0,1,0,0,0,0,1,0,1,0


In [19]:
features = data_dummies.loc[:, 'age':'poutcome_success']
X = features.values
y = data_dummies['y_no'].values
print("X.shape: {}  y.shape: {}".format(X.shape, y.shape))
print(data_dummies.y_no.value_counts())

X.shape: (4119, 63)  y.shape: (4119,)
1    3668
0     451
Name: y_no, dtype: int64


In [20]:
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', LogisticRegression())])
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10]}

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)
grid = GridSearchCV(pipe, param_grid=param_grid, cv=3, return_train_score=False,  scoring="roc_auc")
grid.fit(X_train, y_train)
print("Best parameters: ", grid.best_params_)
print("grid best score, ", grid.best_score_)
print("Test set AUC: {:.2f}".format(grid.score(X_test, y_test)))


Best parameters:  {'classifier__C': 0.01}
grid best score,  0.9383648126404396
Test set AUC: 0.91


In [21]:
#確認用
grid.cv_results_ 

{'mean_fit_time': array([0.01865737, 0.0186549 , 0.02839692, 0.0386885 , 0.04794526]),
 'std_fit_time': array([0.00329498, 0.00376922, 0.00976899, 0.01539907, 0.00777076]),
 'mean_score_time': array([0.00299772, 0.0019997 , 0.00459878, 0.00602341, 0.0016892 ]),
 'std_score_time': array([8.16048190e-04, 7.37000982e-07, 2.97175905e-03, 4.33865388e-03,
        4.76186348e-04]),
 'param_classifier__C': masked_array(data=[0.001, 0.01, 0.1, 1, 10],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'classifier__C': 0.001},
  {'classifier__C': 0.01},
  {'classifier__C': 0.1},
  {'classifier__C': 1},
  {'classifier__C': 10}],
 'split0_test_score': array([0.92583855, 0.9359302 , 0.93226051, 0.92288033, 0.91768472]),
 'split1_test_score': array([0.90608588, 0.92539856, 0.91830258, 0.91121596, 0.90822966]),
 'split2_test_score': array([0.93551007, 0.95378064, 0.95110851, 0.94761491, 0.94777543]),
 'mean_test_score': array([0.92